In [1]:
from transformers import BartForConditionalGeneration, BartTokenizer, DataCollatorForSeq2Seq
from datasets import load_dataset, Dataset, load_metric
import torch

# Define file paths (modify if necessary)
train_file_src = "./train.src.cleaned"
train_file_tgt = "./train.tgt"

completed


In [19]:
def load_text_data(src_file, tgt_file=None,size=1000):
    src_lines = open(src_file, encoding="utf-8").read().strip().split("\n")
    src_lines = src_lines[0:size]
    for line in src_lines:
        line.replace('|||||', ' ')
    if tgt_file:
        tgt_lines = open(tgt_file, encoding="utf-8").read().strip().split("\n")
        tgt_lines = tgt_lines[0:size]
        print(len(src_lines))
        print(len(tgt_lines))
        assert len(src_lines) == len(tgt_lines), "Source and target files should have the same number of lines."
        return {"src": src_lines, "tgt": tgt_lines}
    return {"src": src_lines}

train_data = load_text_data(train_file_src, train_file_tgt)

# Convert to Hugging Face Dataset
train_dataset = Dataset.from_dict(train_data)


1000
1000
completed


In [4]:
from torch.utils.data import DataLoader

tokenizer = BartTokenizer.from_pretrained("facebook/bart-base")

# Preprocess data (tokenization, truncation, padding)
def preprocess_function(examples):
    inputs = tokenizer(examples["src"], truncation=True, padding="max_length", max_length=512)
    if "tgt" in examples:
        with tokenizer.as_target_tokenizer():
            labels = tokenizer(examples["tgt"], truncation=True, padding="max_length", max_length=128)
        inputs["labels"] = labels["input_ids"]
    return inputs

train_dataset = train_dataset.map(preprocess_function, batched=True)
# test_dataset = test_dataset.map(preprocess_function, batched=True)
train_dataset = train_dataset.remove_columns(["src", "tgt"])
for data in train_dataset:
    print(data)
    break

train_dataset.set_format(type="torch")


Map:   0%|          | 0/1000 [00:00<?, ? examples/s]

C:\Users\krishna\AppData\Local\Programs\Python\Python312\Lib\site-packages\transformers\tokenization_utils_base.py:4016: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(


{'input_ids': [0, 18285, 26844, 5178, 28302, 1215, 29146, 5178, 28302, 1215, 29146, 3216, 6, 24, 17, 27, 29, 14, 86, 456, 6, 5450, 4, 85, 17, 27, 29, 5, 78, 273, 9, 5, 353, 6, 77, 13, 65, 655, 12, 2527, 12, 428, 24062, 1151, 5, 3168, 9, 2298, 852, 6, 663, 8, 4326, 852, 32, 70, 14485, 15, 65, 631, 35, 13499, 4, 5178, 28302, 1215, 29146, 5178, 28302, 1215, 29146, 83, 2310, 2935, 15, 5, 121, 4, 104, 4, 4042, 1068, 13, 644, 2323, 5, 22893, 23, 290, 35, 541, 10, 4, 119, 4, 188, 469, 86, 1839, 65, 9, 5, 144, 505, 40617, 15, 141, 5, 866, 24779, 148, 5, 986, 353, 4, 12809, 1635, 32, 13, 23041, 6, 151, 92, 1315, 7, 28, 1412, 6, 309, 7, 9019, 13829, 30, 4614, 1454, 188, 4184, 7948, 6, 1118, 7, 30398, 6, 151, 1315, 355, 11, 902, 4, 20, 5755, 731, 16, 421, 7, 946, 5204, 23, 290, 4, 246, 2153, 5178, 28302, 1215, 29146, 5178, 28302, 1215, 29146, 1398, 23, 1018, 5522, 21956, 6, 52, 17, 27, 890, 28, 1839, 3195, 9765, 137, 8, 71, 5, 414, 20238, 5, 22893, 4, 22079, 481, 7, 9832, 12, 179, 2512, 6, 1241, 

In [5]:
from torch.optim import AdamW
from transformers import get_linear_schedule_with_warmup
train_dataloader = DataLoader(train_dataset, batch_size=10, shuffle=True)


# Define data collator
data_collator = DataCollatorForSeq2Seq(tokenizer, model=BartForConditionalGeneration.from_pretrained("facebook/bart-base"))

# Load BART model
model = BartForConditionalGeneration.from_pretrained("facebook/bart-base")

optimizer = AdamW(model.parameters(), lr=2e-5)
num_epochs = 1  # Adjust number of epochs as needed
num_training_steps = len(train_dataloader) * num_epochs
scheduler = get_linear_schedule_with_warmup(optimizer, num_warmup_steps=0, num_training_steps=num_training_steps)

# Define training function
def train(model, optimizer, scheduler, data_collator, train_dataloader, num_epochs):
    model.train()
    for epoch in range(num_epochs):
        i=0
        for batch in train_dataloader:
            batch = {k: v.to(model.device) for k, v in batch.items()}
            outputs = model(**batch)
            loss = outputs.loss
            loss.backward()
            optimizer.step()
            scheduler.step()
            optimizer.zero_grad()
            print(i)
            i+=1
            print(f"Epoch {epoch + 1}, Loss: {loss.item()}")

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

# Train the model
train(model, optimizer, scheduler, data_collator, train_dataloader, num_epochs)

print("completed")

0
Epoch 1, Loss: 3.849430799484253
1
Epoch 1, Loss: 3.652371644973755
2
Epoch 1, Loss: 4.0699052810668945
3
Epoch 1, Loss: 3.362576961517334
4
Epoch 1, Loss: 3.451690673828125
5
Epoch 1, Loss: 3.539350986480713
6
Epoch 1, Loss: 3.573878526687622
7
Epoch 1, Loss: 3.660527467727661
8
Epoch 1, Loss: 2.964665412902832
9
Epoch 1, Loss: 3.4175338745117188
10
Epoch 1, Loss: 3.120920419692993
11
Epoch 1, Loss: 3.5812206268310547
12
Epoch 1, Loss: 3.3877320289611816
13
Epoch 1, Loss: 3.3772664070129395
14
Epoch 1, Loss: 3.6474785804748535
15
Epoch 1, Loss: 3.1023521423339844
16
Epoch 1, Loss: 3.074369192123413
17
Epoch 1, Loss: 3.6556785106658936
18
Epoch 1, Loss: 3.1568048000335693
19
Epoch 1, Loss: 3.057084321975708
20
Epoch 1, Loss: 3.375124454498291
21
Epoch 1, Loss: 3.566352128982544
22
Epoch 1, Loss: 3.172957420349121
23
Epoch 1, Loss: 3.26172137260437
24
Epoch 1, Loss: 3.1798205375671387
25
Epoch 1, Loss: 3.230856418609619
26
Epoch 1, Loss: 3.6109395027160645
27
Epoch 1, Loss: 3.59329366

In [8]:
model.save_pretrained("finetuned_bart_summarization1")
tokenizer.save_pretrained("finetuned_bart_summarization1")
print("completed")

Some non-default generation parameters are set in the model config. These should go into a GenerationConfig file (https://huggingface.co/docs/transformers/generation_strategies#save-a-custom-decoding-strategy-with-your-model) instead. This warning will be raised to an exception in v4.41.
Non-default generation parameters: {'early_stopping': True, 'num_beams': 4, 'no_repeat_ngram_size': 3, 'forced_bos_token_id': 0, 'forced_eos_token_id': 2}


completed


In [17]:
test_file_src = "./test.src.cleaned"
test_file_tgt = "./test.tgt"

In [30]:
test_data = load_text_data(test_file_src, test_file_tgt,10)
test_dataset = Dataset.from_dict(test_data)

10
10


In [31]:


test_dataset = test_dataset.map(preprocess_function, batched=True)

test_dataset = test_dataset.remove_columns(["src", "tgt"])

Map:   0%|          | 0/10 [00:00<?, ? examples/s]

C:\Users\krishna\AppData\Local\Programs\Python\Python312\Lib\site-packages\transformers\tokenization_utils_base.py:4016: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(


In [32]:
test_dataset.set_format(type="torch")

In [33]:
test_dataloader = DataLoader(test_dataset, batch_size=10)

In [34]:
# Evaluate the model
def evaluate(model, test_dataloader, tokenizer):
    model.eval()
    predictions, references = [], []
    for batch in test_dataloader:
        inputs = batch["input_ids"].to(model.device)
        attention_mask = batch["attention_mask"].to(model.device)
        with torch.no_grad():
            outputs = model.generate(inputs, attention_mask=attention_mask)
        decoded_preds = tokenizer.batch_decode(outputs, skip_special_tokens=True)
        decoded_labels = tokenizer.batch_decode(batch["labels"], skip_special_tokens=True)
        predictions.extend(decoded_preds)
        references.extend(decoded_labels)
    return predictions, references

# Load the model
model = BartForConditionalGeneration.from_pretrained("finetuned_bart_summarization1")
tokenizer = BartTokenizer.from_pretrained("finetuned_bart_summarization1")

# Prepare the test dataloader
test_dataloader = DataLoader(test_dataset, batch_size=10)

# Get predictions and references
predictions, references = evaluate(model, test_dataloader, tokenizer)
print("rouge started")
# Load the metric
rouge = load_metric("rouge", trust_remote_code=True)

# Compute the metrics
results = rouge.compute(predictions=predictions, references=references, use_stemmer=True)
for key, value in results.items():
    print(f"{key}: {value.mid.fmeasure:.4f}")

print("Evaluation completed")


C:\Users\krishna\AppData\Local\Programs\Python\Python312\Lib\site-packages\transformers\generation\utils.py:1249: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


0
rouge started
rouge1: 0.1356
rouge2: 0.0341
rougeL: 0.1053
rougeLsum: 0.1047
Evaluation completed


In [35]:
from transformers import BartForConditionalGeneration, BartTokenizer
import torch

# Load the fine-tuned model and tokenizer
model_name = "finetuned_bart_summarization1"
model = BartForConditionalGeneration.from_pretrained(model_name)
tokenizer = BartTokenizer.from_pretrained(model_name)

# Ensure the model is in evaluation mode
model.eval()

# Set the device to GPU if available, otherwise CPU
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

def summarize_article(article_text):
    # Tokenize the input text
    inputs = tokenizer(article_text, return_tensors="pt", max_length=512, truncation=True)
    input_ids = inputs.input_ids.to(device)
    attention_mask = inputs.attention_mask.to(device)
    
    # Generate summary
    with torch.no_grad():
        summary_ids = model.generate(input_ids, attention_mask=attention_mask, max_length=128, num_beams=4, length_penalty=2.0, early_stopping=True)
    
    # Decode the summary
    summary = tokenizer.decode(summary_ids[0], skip_special_tokens=True)
    return summary

# Example usage
article_text = """
Based on a ‘real’ story, the hit John Travolta film Saturday Night Fever became the prism through which the world viewed disco. 
Twenty years later it was revealed that the actual inspiration was a British mod called Chris... NEWLINE_CHAR NEWLINE_CHAR Picture it: 
a writer pens a magazine article and it’s an instant sensation. Producers come calling, he sells the rights for tens of thousands, 
the tabloids give him a nickname, acquaintances greet him as a friend, cheques flood in, he attends the premiere of his film in 
Los Angeles with a famous disco singer on his arm. It’s glitzy, it’s glam, it’s Hollywood, baby. But as he makes his way through 
the frenzy outside the theatre, through security, paparazzi and screaming teenage girls, he is filled with moral panic. 
Why? NEWLINE_CHAR NEWLINE_CHAR Saturday Night Fever was released in 1977, and has since grossed $285m worldwide. 
The soundtrack became one of the bestselling film album of all time after staying at No 1 for 24 consecutive weeks, 
reinvigorating the Bee Gees’ career, and its star, John Travolta, became one of the youngest actors to be nominated for 
the best actor Oscar. Decades on, not many remember that the phenomenon was down to one man: Northern Irish rock critic 
Nik Cohn and his report of 7 June, 1976 for New York magazine, Tribal Rites of the New Saturday Nights, which was published 
40 years ago this month. NEWLINE_CHAR NEWLINE_CHAR Cohn was the author of a number of books including the 1969 rock history 
Awopbopaloobop Alopbamboom. A protege of swinging London, he partied with rock stars, joined the Who on tour, and is said by 
some to have been instrumental in the genesis of Bowie’s Ziggy Stardust. He had his portrait shot by Iain Macmillan – the 
photographer behind the Beatles’ Abbey Road cover – and from the age of 18 was contributing briefings about mods and rockers to the 
Observer. After crossing the Atlantic and signing on with New York magazine in 1975, the writer, who came to feel disenchanted with 
the establishment music business, persuaded the mag’s founder and editor Clay Felker to let him document disco – a new, largely ethnic, 
largely gay underground trend that had taken over parts of New York City. NEWLINE_CHAR NEWLINE_CHAR The writer was painfully aware that 
everything Fever had brought him – the fame, the fortune – was the result of a lie NEWLINE_CHAR NEWLINE_CHAR The result was the profile 
of an “ultimate Face”. Vincent, a young Italian-American worked in a hardware store during the week and partied at a disco club called 
2001 Odyssey on the weekend. Vincent “was the very best dancer in Bay Ridge … he owned 14 floral shirts, five suits, eight pairs of 
shoes, three overcoats, and had appeared on American Bandstand”. He and his friends knew nothing of flower power, Bob Dylan or Ken Kesey. 
They were opulent but poor, proud but shy. “The new generation takes few risks,” Cohn wrote. “It goes through high school, obedient; 
graduates, looks for a job, saves and plans. Endures. And once a week, on Saturday night, its one great moment of release, it explodes.” 
The intro declared: “Everything described in this article is factual and was either witnessed by me or told to me directly by the people 
involved. Only the names of the main characters have been changed.” NEWLINE_CHAR NEWLINE_CHAR Facebook Twitter Pinterest Watch a trailer 
for Saturday Night Fever NEWLINE_CHAR NEWLINE_CHAR The rest is cinema history: film rights were sold to producer Robert Stigwood, who 
had just signed a three-picture deal with a young TV actor called John Travolta. Screenwriter Norman Wexler transformed Vincent into Tony 
Manero. So unprecedented was the fanfare that when Stigwood’s 23-year-old assistant Kevin McCormick traipsed through Los Angeles looking 
for a director, one agent, according to Vanity Fair, told him, “Kid, my directors do movies. They don’t do magazine articles.” Director 
John Badham had no such qualms, and in December 1977 his movie took $11m in its first 11 days and Travolta became an overnight sensation. 
NEWLINE_CHAR NEWLINE_CHAR Twenty years later came a bombshell. In December 1997 New York magazine published an article in which Cohn confessed 
that there never was a Vincent. There was no “Lisa”, “Billy”, “John James”, “Lorraine” or “Donna” either. While 2001 Odyssey existed, 
it wasn’t the way the writer described it in 1976. The whole scene of disco-loving Italians, as mythologised in Saturday Night Fever, 
was exaggerated. The most bizarre detail was that his disco protagonists were in fact based on mods Cohn had known in London. 
The writer was “painfully aware” that everything Fever had brought him – the fame, the fortune – was the result of a lie. 
NEWLINE_CHAR NEWLINE_CHAR The real story went like this: in 1976 Cohn met a disco dancer named Tu Sweet, who introduced him 
to the clubs of New York, including one in Bay Ridge called 2001 Odyssey. One night the two trawled through the underbelly of New York 
– a land of auto shops, transmission specialists and alignment centres – to find the place. A drunken brawl was in progress and as Cohn 
opened the cab door one of the guys reeled over the gutter and threw up over his trouser leg. So he just upped and returned to the safe 
comforts of Manhattan. NEWLINE_CHAR NEWLINE_CHAR One image stayed with the writer, though, that of a figure in flared crimson pants and 
a black body shirt standing in the doorway of the club and calmly watching the action. There was a style about him, Cohn said, a sense of 
his own specialness that reminded the writer of a teen gang in his hometown of Derry and a mod named Chris he’d met in London in 1965. 
NEWLINE_CHAR NEWLINE_CHAR How we made Sister Sledge’s We Are Family Read more NEWLINE_CHAR NEWLINE_CHAR When Cohn went back to Odyssey 
he didn’t see the young man in the doorway again. “Plus, I made a lousy interviewer,” he wrote. “I knew nothing about this world, and it showed. 
Quite literally, I didn’t speak the language. So I faked it. I conjured up the story of the figure in the doorway, and named him Vincent. 
Taking all I knew about the snake-charmer in Derry and, more especially, about Chris the mod in London, I translated them as best I 
could to Brooklyn. Then I went back to Bay Ridge in daylight and noted the major landmarks. I walked some streets, went into a couple of 
stores. Studied the clothes, the gestures, the walks. Imagined how it would feel to burn up, all caged energies, with no outlet but the 
dancefloor and the rituals of Saturday night. Finally, I wrote it all up. And presented it as fact.” NEWLINE_CHAR NEWLINE_CHAR Facebook 
Twitter Pinterest A premiere party for Saturday Night Fever with NEWLINE_CHAR NEWLINE_CHAR John Travolta, right, next to a bearded Nik Cohn, 
Travolta’s mother below them, and producer Robert Stigwood (far left). Photograph: Ron Galella/WireImage NEWLINE_CHAR NEWLINE_CHAR So 
how did he get away with one of the most daring acts of journalistic forgery? While Tribal Rites reads like a novelisation, it must be 
understood in the context of the time it was written: the tail-end of the era of New Journalism, where writers used literary techniques 
and a subjective perspective to present fact as fiction. It followed similar works by the likes of Tom Wolfe, Gay Talese, Truman Capote, 
Hunter S Thompson and Norman Mailer. NEWLINE_CHAR NEWLINE_CHAR When I first approached Cohn for this article, he said he no longer wanted 
to discuss the topic. But after several back-and-forth emails, he did say that he doubted any magazine would publish the Tribal Rites piece 
today. “It reads to me as obvious fiction, albeit based on observation and some knowledge of disco culture. No way could it sneak past 
customs now. In the 60s and 70s, the line between fact and fiction was blurry. Many magazine writers used fictional techniques to tell 
supposedly factual stories. No end of liberties were taken. Few editors asked tough questions. For the most part it was a case of ‘don’t 
ask, don’t tell’. NEWLINE_CHAR NEWLINE_CHAR “Magazine writing then was basically a boys’ club. There was a lot of wretched excess. Along 
with some great writing came reams of self-indulgent bollocks. Tribal Rites being fiction was never a great secret. I remember once, at the 
end of a long night, blurting out to a publisher that the story was made up. ‘You don’t say,’ the publisher drawled. ‘And Liberace is gay.’” 
NEWLINE_CHAR NEWLINE_CHAR For context, Gay Talese, now a bestselling author and one of the pioneers of new journalism and author 
of ‘Frank Sinatra Has a Cold’, explains that his intention as a young journalist was to write short stories using real characters. 
“I have always been inspired by great short story writers, the first being the French writer Guy de Maupassant,” he said. 
“Later I also began reading short stories by famous novelists – F Scott Fitzgerald, Ernest Hemingway and more contemporary 
writers such as John O’Hara, Irwin Shaw, Carson McCullers, John Cheever and others. My newspaper articles were all written as if 
short stories: there was scene-setting, dialogue, much description of people and places.” But while the articles were presented as 
stories, they were never fictionalised. “Nothing was invented, all the names of the characters were real, and verifiably truthful.” NEWLINE_CHAR NEWLINE_CHAR Caroline Miller, who edited New York magazine at the time of Cohn’s confession, said her predecessor, Felker, wouldn’t have published Tribal Rites if he thought it fabricated. “That said, remember that 70s Brooklyn was a foreign country to most New York magazine editors,” she told me. “It wasn’t cool, and some of them had probably never been there – even to Brooklyn Heights, which was Norman Mailer territory. So they may not have had good radar for credibility. Also, after the zeitgeisty opening about the blue-collar disco tribe, [Tribal Rites] is all narrative, and that much narrative detail tends to read as real. Conversation in cars. What Vincent was thinking as he looked in the mirror…” NEWLINE_CHAR NEWLINE_CHAR Miller and her team published Cohn’s admission because it was newsworthy. “Here’s a guy basically bragging about fabricating a legendary story and getting away with it,” she said. “And it certainly added to our understanding of pop culture myth-making – the idea that a mashup of people and scenes Nik had collected on both side of the Atlantic could go essentially unchallenged, and have such staying power.” NEWLINE_CHAR NEWLINE_CHAR Facebook Twitter Pinterest Tribal Rites of the New Saturday Night by Nik Cohn as featured on a 1976 New York magazine cover. NEWLINE_CHAR NEWLINE_CHAR Cohn has said that were it not for Jim McMullan’s accompanying illustrations the piece might never have seen daylight. McMullan based these on the photographs he took in 2001 Odyssey but, fundamentally, he never met Cohn’s protagonist. “I went to the club twice and moved around, taking my photos without interacting much with any of the patrons,” McMullan recalled. “Nik took a different path through the crowds so we didn’t exchange notes. NEWLINE_CHAR NEWLINE_CHAR It’s a universal story... working-class kids going out and getting off their heads to American black music Bill Brewster NEWLINE_CHAR NEWLINE_CHAR “I finished my paintings several weeks before Nik finished his story so I wasn’t really reacting to how he saw the scene at the club. It did seem like an amazingly dramatic story arc and the kind of ‘working-class’ story he was already famous for.” NEWLINE_CHAR NEWLINE_CHAR It so happened that the design director of New York magazine, Milton Glaser, was amazed by the reportage style of the paintings. “Clay also came to see the work that way... I suspect that because the art was all ready to go before Nik finished writing, it put some pressure on him to get it written. Had the paintings not already impressed Milton and Clay, I suppose it might have been easier to scuttle the whole project.” NEWLINE_CHAR NEWLINE_CHAR Did McMullan’s art indicate a truth to the piece? While Cohn’s descriptions of the club’s “Faces” were based on the working class in England, they weren’t entirely off the mark. NEWLINE_CHAR NEWLINE_CHAR “Just like the Italian-Americans, the mods shopped for that perfect shirt,” Colleen “Cosmo” Murphy, an American radio broadcaster, DJ, producer and founder of Classic Album Sundays, explained to me over coffee in London one morning. “It was about looking like they were better off than they really were, sticking their money into things like music and clothes. They bought records, certain types of trousers, certain types of jackets. It was an attitude. NEWLINE_CHAR NEWLINE_CHAR “It’s that culture of people who have their regular job during the week and live to go out on the weekend. A blue-collar worker in Brooklyn isn’t going to be defined by their job. Who they really are is who they are when they go out. Just like a mod. A mod might be bricklayer, but they’re a mod.” NEWLINE_CHAR NEWLINE_CHAR Bill Brewster, former editor of Mixmag USA and founder of Djhistory.com, echoed this: “It’s a universal story that had been going on for decades even before there were DJs playing records,” he said. “Working-class kids going out on a Friday and Saturday and getting off their heads to American black music. It was happening in the cellars of Paris during the occupation in terms of jazz records. Those archetypes, even though Cohn based them on people in London, were obviously happening in New York as well. I think it was an educated guess on his part, and a correct one.” NEWLINE_CHAR NEWLINE_CHAR Facebook Twitter Pinterest The multi-million-selling soundtrack album featured hits from the Bee Gees and topped the US chart for 24 consecutive weeks. NEWLINE_CHAR NEWLINE_CHAR Disco as a genre and culture had already been gathering pace in New York so Saturday Night Fever was just the tipping point of something that had been going on for a while. “Even before disco was officially called disco, you had David Mancuso’s loft parties, in the late 60s, in downtown Manhattan, where he played danceable acid-rock, R&B, mixing it all up,” said Murphy, who has collaborated with Mancuso in the past. NEWLINE_CHAR NEWLINE_CHAR “They were all about integrating different kinds of people, whether it was class, race, sexual orientation. People like David Morales and Larry Levan were going as kids. At the same time Francis Grasso became the first DJ to start mixing records together with two turntables. In the mid-70s these other clubs started rising – Studio 54, which was the glitzy manhattan club, where Andy Warhol, Grace Jones and Liza Minelli hung out, and places like 2001 Odyssey, which were for the working-class Brooklynites.” NEWLINE_CHAR NEWLINE_CHAR But in the end it was Cohn’s article and Saturday Night Fever that gave the decade its cultural identity. “As a child I was living in Massachusetts, in a white suburban New England small town, listening to rock and pop ,” Murphy said. “Then this article happened, the movie came out and disco blew up across white America overnight. Me and my friends would dance to the soundtrack at slumber parties. I had aunts and uncles who were taking disco-dancing lessons.” NEWLINE_CHAR NEWLINE_CHAR Build, baby, build: when radical architects did disco Read more NEWLINE_CHAR NEWLINE_CHAR Then came the backlash. DJ Steve Dahl headed up a Disco Demolition Night in Chicago in July 1979. People wore “disco sucks” T-shirts. The Bee Gees became cheesy, Chic became cheesy, and by the 80s disco was a dirty word. “Saturday Night Fever was probably the thing that killed off disco in the end,” Murphy said. NEWLINE_CHAR NEWLINE_CHAR Cohn, now 70, lives in Ghent, New York. His life reads like a blockbuster of its own – after Tribal Rites he continued writing, true stories mostly, and in 1983 was arrested for conspiring to import millions of dollars worth of heroin and cocaine into the US. The more serious charges were dropped and Cohn was given five years’ probation for possession. His life, the writer then realised, had been unravelling and it was time for a change. “Why did I decide to come clean in 1997? It simply felt like time,” he told me. “What seemed OK to me when I was young and stoned no longer sat right. Accountability, let’s say.” NEWLINE_CHAR NEWLINE_CHAR Cohn has always maintained that what was genuine was the staying power of Saturday Night Fever itself. That central figure, with all his grace, energy and passion. A nobody who once a week was a somebody. “Tribal Rites is about identity,” he said. “Finding a place in the world where you can shine. What still resonates, to me at least, is the sense of yearning. If I was writing the story today, Vincent might be trans…” ||||| Based on a ‘real’ story, the hit John Travolta film Saturday Night Fever became the prism through which the world viewed disco. Twenty years later it was revealed that the actual inspiration was a British mod called Chris... NEWLINE_CHAR NEWLINE_CHAR Picture it: a writer pens a magazine article and it’s an instant sensation. Producers come calling, he sells the rights for tens of thousands, the tabloids give him a nickname, acquaintances greet him as a friend, cheques flood in, he attends the premiere of his film in Los Angeles with a famous disco singer on his arm. It’s glitzy, it’s glam, it’s Hollywood, baby. But as he makes his way through the frenzy outside the theatre, through security, paparazzi and screaming teenage girls, he is filled with moral panic. Why? NEWLINE_CHAR NEWLINE_CHAR Saturday Night Fever was released in 1977, and has since grossed $285m worldwide. The soundtrack became one of the bestselling film album of all time after staying at No 1 for 24 consecutive weeks, reinvigorating the Bee Gees’ career, and its star, John Travolta, became one of the youngest actors to be nominated for the best actor Oscar. Decades on, not many remember that the phenomenon was down to one man: Northern Irish rock critic Nik Cohn and his report of 7 June, 1976 for New York magazine, Tribal Rites of the New Saturday Nights, which was published 40 years ago this month. NEWLINE_CHAR NEWLINE_CHAR Cohn was the author of a number of books including the 1969 rock history Awopbopaloobop Alopbamboom. A protege of swinging London, he partied with rock stars, joined the Who on tour, and is said by some to have been instrumental in the genesis of Bowie’s Ziggy Stardust. He had his portrait shot by Iain Macmillan – the photographer behind the Beatles’ Abbey Road cover – and from the age of 18 was contributing briefings about mods and rockers to the Observer. After crossing the Atlantic and signing on with New York magazine in 1975, the writer, who came to feel disenchanted with the establishment music business, persuaded the mag’s founder and editor Clay Felker to let him document disco – a new, largely ethnic, largely gay underground trend that had taken over parts of New York City. NEWLINE_CHAR NEWLINE_CHAR The writer was painfully aware that everything Fever had brought him – the fame, the fortune – was the result of a lie NEWLINE_CHAR NEWLINE_CHAR The result was the profile of an “ultimate Face”. Vincent, a young Italian-American worked in a hardware store during the week and partied at a disco club called 2001 Odyssey on the weekend. Vincent “was the very best dancer in Bay Ridge … he owned 14 floral shirts, five suits, eight pairs of shoes, three overcoats, and had appeared on American Bandstand”. He and his friends knew nothing of flower power, Bob Dylan or Ken Kesey. They were opulent but poor, proud but shy. “The new generation takes few risks,” Cohn wrote. “It goes through high school, obedient; graduates, looks for a job, saves and plans. Endures. And once a week, on Saturday night, its one great moment of release, it explodes.” The intro declared: “Everything described in this article is factual and was either witnessed by me or told to me directly by the people involved. Only the names of the main characters have been changed.” NEWLINE_CHAR NEWLINE_CHAR Facebook Twitter Pinterest Watch a trailer for Saturday Night Fever NEWLINE_CHAR NEWLINE_CHAR The rest is cinema history: film rights were sold to producer Robert Stigwood, who had just signed a three-picture deal with a young TV actor called John Travolta. Screenwriter Norman Wexler transformed Vincent into Tony Manero. So unprecedented was the fanfare that when Stigwood’s 23-year-old assistant Kevin McCormick traipsed through Los Angeles looking for a director, one agent, according to Vanity Fair, told him, “Kid, my directors do movies. They don’t do magazine articles.” Director John Badham had no such qualms, and in December 1977 his movie took $11m in its first 11 days and Travolta became an overnight sensation. NEWLINE_CHAR NEWLINE_CHAR Twenty years later came a bombshell. In December 1997 New York magazine published an article in which Cohn confessed that there never was a Vincent. There was no “Lisa”, “Billy”, “John James”, “Lorraine” or “Donna” either. While 2001 Odyssey existed, it wasn’t the way the writer described it in 1976. The whole scene of disco-loving Italians, as mythologised in Saturday Night Fever, was exaggerated. The most bizarre detail was that his disco protagonists were in fact based on mods Cohn had known in London. The writer was “painfully aware” that everything Fever had brought him – the fame, the fortune – was the result of a lie. NEWLINE_CHAR NEWLINE_CHAR The real story went like this: in 1976 Cohn met a disco dancer named Tu Sweet, who introduced him to the clubs of New York, including one in Bay Ridge called 2001 Odyssey. One night the two trawled through the underbelly of New York – a land of auto shops, transmission specialists and alignment centres – to find the place. A drunken brawl was in progress and as Cohn opened the cab door one of the guys reeled over the gutter and threw up over his trouser leg. So he just upped and returned to the safe comforts of Manhattan. NEWLINE_CHAR NEWLINE_CHAR One image stayed with the writer, though, that of a figure in flared crimson pants and a black body shirt standing in the doorway of the club and calmly watching the action. There was a style about him, Cohn said, a sense of his own specialness that reminded the writer of a teen gang in his hometown of Derry and a mod named Chris he’d met in London in 1965. NEWLINE_CHAR NEWLINE_CHAR How we made Sister Sledge’s We Are Family Read more NEWLINE_CHAR NEWLINE_CHAR When Cohn went back to Odyssey he didn’t see the young man in the doorway again. “Plus, I made a lousy interviewer,” he wrote. “I knew nothing about this world, and it showed. Quite literally, I didn’t speak the language. So I faked it. I conjured up the story of the figure in the doorway, and named him Vincent. Taking all I knew about the snake-charmer in Derry and, more especially, about Chris the mod in London, I translated them as best I could to Brooklyn. Then I went back to Bay Ridge in daylight and noted the major landmarks. I walked some streets, went into a couple of stores. Studied the clothes, the gestures, the walks. Imagined how it would feel to burn up, all caged energies, with no outlet but the dancefloor and the rituals of Saturday night. Finally, I wrote it all up. And presented it as fact.” NEWLINE_CHAR NEWLINE_CHAR Facebook Twitter Pinterest A premiere party for Saturday Night Fever with NEWLINE_CHAR NEWLINE_CHAR John Travolta, right, next to a bearded Nik Cohn, Travolta’s mother below them, and producer Robert Stigwood (far left). Photograph: Ron Galella/WireImage NEWLINE_CHAR NEWLINE_CHAR So how did he get away with one of the most daring acts of journalistic forgery? While Tribal Rites reads like a novelisation, it must be understood in the context of the time it was written: the tail-end of the era of New Journalism, where writers used literary techniques and a subjective perspective to present fact as fiction. It followed similar works by the likes of Tom Wolfe, Gay Talese, Truman Capote, Hunter S Thompson and Norman Mailer. NEWLINE_CHAR NEWLINE_CHAR When I first approached Cohn for this article, he said he no longer wanted to discuss the topic. But after several back-and-forth emails, he did say that he doubted any magazine would publish the Tribal Rites piece today. “It reads to me as obvious fiction, albeit based on observation and some knowledge of disco culture. No way could it sneak past customs now. In the 60s and 70s, the line between fact and fiction was blurry. Many magazine writers used fictional techniques to tell supposedly factual stories. No end of liberties were taken. Few editors asked tough questions. For the most part it was a case of ‘don’t ask, don’t tell’. NEWLINE_CHAR NEWLINE_CHAR “Magazine writing then was basically a boys’ club. There was a lot of wretched excess. Along with some great writing came reams of self-indulgent bollocks. Tribal Rites being fiction was never a great secret. I remember once, at the end of a long night, blurting out to a publisher that the story was made up. ‘You don’t say,’ the publisher drawled. ‘And Liberace is gay.’” NEWLINE_CHAR NEWLINE_CHAR For context, Gay Talese, now a bestselling author and one of the pioneers of new journalism and author of ‘Frank Sinatra Has a Cold’, explains that his intention as a young journalist was to write short stories using real characters. “I have always been inspired by great short story writers, the first being the French writer Guy de Maupassant,” he said. “Later I also began reading short stories by famous novelists – F Scott Fitzgerald, Ernest Hemingway and more contemporary writers such as John O’Hara, Irwin Shaw, Carson McCullers, John Cheever and others. My newspaper articles were all written as if short stories: there was scene-setting, dialogue, much description of people and places.” But while the articles were presented as stories, they were never fictionalised. “Nothing was invented, all the names of the characters were real, and verifiably truthful.” NEWLINE_CHAR NEWLINE_CHAR Caroline Miller, who edited New York magazine at the time of Cohn’s confession, said her predecessor, Felker, wouldn’t have published Tribal Rites if he thought it fabricated. “That said, remember that 70s Brooklyn was a foreign country to most New York magazine editors,” she told me. “It wasn’t cool, and some of them had probably never been there – even to Brooklyn Heights, which was Norman Mailer territory. So they may not have had good radar for credibility. Also, after the zeitgeisty opening about the blue-collar disco tribe, [Tribal Rites] is all narrative, and that much narrative detail tends to read as real. Conversation in cars. What Vincent was thinking as he looked in the mirror…” NEWLINE_CHAR NEWLINE_CHAR Miller and her team published Cohn’s admission because it was newsworthy. “Here’s a guy basically bragging about fabricating a legendary story and getting away with it,” she said. “And it certainly added to our understanding of pop culture myth-making – the idea that a mashup of people and scenes Nik had collected on both side of the Atlantic could go essentially unchallenged, and have such staying power.” NEWLINE_CHAR NEWLINE_CHAR Facebook Twitter Pinterest Tribal Rites of the New Saturday Night by Nik Cohn as featured on a 1976 New York magazine cover. NEWLINE_CHAR NEWLINE_CHAR Cohn has said that were it not for Jim McMullan’s accompanying illustrations the piece might never have seen daylight. McMullan based these on the photographs he took in 2001 Odyssey but, fundamentally, he never met Cohn’s protagonist. “I went to the club twice and moved around, taking my photos without interacting much with any of the patrons,” McMullan recalled. “Nik took a different path through the crowds so we didn’t exchange notes. NEWLINE_CHAR NEWLINE_CHAR It’s a universal story... working-class kids going out and getting off their heads to American black music Bill Brewster NEWLINE_CHAR NEWLINE_CHAR “I finished my paintings several weeks before Nik finished his story so I wasn’t really reacting to how he saw the scene at the club. It did seem like an amazingly dramatic story arc and the kind of ‘working-class’ story he was already famous for.” NEWLINE_CHAR NEWLINE_CHAR It so happened that the design director of New York magazine, Milton Glaser, was amazed by the reportage style of the paintings. “Clay also came to see the work that way... I suspect that because the art was all ready to go before Nik finished writing, it put some pressure on him to get it written. Had the paintings not already impressed Milton and Clay, I suppose it might have been easier to scuttle the whole project.” NEWLINE_CHAR NEWLINE_CHAR Did McMullan’s art indicate a truth to the piece? While Cohn’s descriptions of the club’s “Faces” were based on the working class in England, they weren’t entirely off the mark. NEWLINE_CHAR NEWLINE_CHAR “Just like the Italian-Americans, the mods shopped for that perfect shirt,” Colleen “Cosmo” Murphy, an American radio broadcaster, DJ, producer and founder of Classic Album Sundays, explained to me over coffee in London one morning. “It was about looking like they were better off than they really were, sticking their money into things like music and clothes. They bought records, certain types of trousers, certain types of jackets. It was an attitude. NEWLINE_CHAR NEWLINE_CHAR “It’s that culture of people who have their regular job during the week and live to go out on the weekend. A blue-collar worker in Brooklyn isn’t going to be defined by their job. Who they really are is who they are when they go out. Just like a mod. A mod might be bricklayer, but they’re a mod.” NEWLINE_CHAR NEWLINE_CHAR Bill Brewster, former editor of Mixmag USA and founder of Djhistory.com, echoed this: “It’s a universal story that had been going on for decades even before there were DJs playing records,” he said. “Working-class kids going out on a Friday and Saturday and getting off their heads to American black music. It was happening in the cellars of Paris during the occupation in terms of jazz records. Those archetypes, even though Cohn based them on people in London, were obviously happening in New York as well. I think it was an educated guess on his part, and a correct one.” NEWLINE_CHAR NEWLINE_CHAR Facebook Twitter Pinterest The multi-million-selling soundtrack album featured hits from the Bee Gees and topped the US chart for 24 consecutive weeks. NEWLINE_CHAR NEWLINE_CHAR Disco as a genre and culture had already been gathering pace in New York so Saturday Night Fever was just the tipping point of something that had been going on for a while. “Even before disco was officially called disco, you had David Mancuso’s loft parties, in the late 60s, in downtown Manhattan, where he played danceable acid-rock, R&B, mixing it all up,” said Murphy, who has collaborated with Mancuso in the past. NEWLINE_CHAR NEWLINE_CHAR “They were all about integrating different kinds of people, whether it was class, race, sexual orientation. People like David Morales and Larry Levan were going as kids. At the same time Francis Grasso became the first DJ to start mixing records together with two turntables. In the mid-70s these other clubs started rising – Studio 54, which was the glitzy manhattan club, where Andy Warhol, Grace Jones and Liza Minelli hung out, and places like 2001 Odyssey, which were for the working-class Brooklynites.” NEWLINE_CHAR NEWLINE_CHAR But in the end it was Cohn’s article and Saturday Night Fever that gave the decade its cultural identity. “As a child I was living in Massachusetts, in a white suburban New England small town, listening to rock and pop ,” Murphy said. “Then this article happened, the movie came out and disco blew up across white America overnight. Me and my friends would dance to the soundtrack at slumber parties. I had aunts and uncles who were taking disco-dancing lessons.” NEWLINE_CHAR NEWLINE_CHAR Build, baby, build: when radical architects did disco Read more NEWLINE_CHAR NEWLINE_CHAR Then came the backlash. DJ Steve Dahl headed up a Disco Demolition Night in Chicago in July 1979. People wore “disco sucks” T-shirts. The Bee Gees became cheesy, Chic became cheesy, and by the 80s disco was a dirty word. “Saturday Night Fever was probably the thing that killed off disco in the end,” Murphy said. NEWLINE_CHAR NEWLINE_CHAR Cohn, now 70, lives in Ghent, New York. His life reads like a blockbuster of its own – after Tribal Rites he continued writing, true stories mostly, and in 1983 was arrested for conspiring to import millions of dollars worth of heroin and cocaine into the US. The more serious charges were dropped and Cohn was given five years’ probation for possession. His life, the writer then realised, had been unravelling and it was time for a change. “Why did I decide to come clean in 1997? It simply felt like time,” he told me. “What seemed OK to me when I was young and stoned no longer sat right. Accountability, let’s say.” NEWLINE_CHAR NEWLINE_CHAR Cohn has always maintained that what was genuine was the staying power of Saturday Night Fever itself. That central figure, with all his grace, energy and passion. A nobody who once a week was a somebody. “Tribal Rites is about identity,” he said. “Finding a place in the world where you can shine. What still resonates, to me at least, is the sense of yearning. If I was writing the story today, Vincent might be trans…”
"""

summary = summarize_article(article_text)
print("Summary:")
print(summary)


Summary:
– John Travolta’s film Saturday Night Fever was a smash hit in 1977, and has since grossed $285m worldwide. According to the New York Times, the film was inspired by a British mod called Chris. Twenty-four years later it was revealed that the actual inspiration was a British Mod called Chris...Twenty years later, the Times reports that the mod was named Chris. The mod is named Chris, and the film is based on his experience in Los Angeles. "The mod writes a magazine article about the film, and it's an instant sensation," writes the Times. "He writes a


In [18]:
!pip install absl-py rouge-score


[notice] A new release of pip is available: 24.0 -> 24.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip



  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'done'
  Preparing metadata (pyproject.toml): started
  Preparing metadata (pyproject.toml): finished with status 'done'
   ---------------------------------------- 0.0/133.7 kB ? eta -:--:--
   --- ------------------------------------ 10.2/133.7 kB ? eta -:--:--
   ----------------- --------------------- 61.4/133.7 kB 656.4 kB/s eta 0:00:01
   -------------------------------------  133.1/133.7 kB 983.0 kB/s eta 0:00:01
   -------------------------------------- 133.7/133.7 kB 875.6 kB/s eta 0:00:00
  Created wheel for rouge-score: filename=rouge_score-0.1.2-py3-none-any.whl size=24974 sha256=cf8c6560c64f49f35d5dc1eec5010fad0231e24407682a051d7178b0339cb44a
  Stored in directory: c:\users\krishna\appdata\local\pip\cache\wheels\85\9d\af\01feefbe7d55ef5468796f0c68225b6788e85d9d0a281e